In [1]:
import netCDF4
import numpy as np
import numpy.ma as ma

In [2]:
anzda_recon_txt = np.loadtxt('/d4/yzhou/Drought_atlas/ANZDA/anzda-recon.txt',skiprows=1)

In [3]:
time_np = anzda_recon_txt[:,0]
time_ma = ma.asarray(time_np)

In [4]:
# delete time from anzda_recon_txt, which is the first element of each row
anzda_recon_txt = [row[1:] for row in anzda_recon_txt]

In [5]:
anzda_pdsi_lon_lat_pair_txt = np.loadtxt('/d4/yzhou/Drought_atlas/ANZDA/anzda-pdsi-xy.txt',usecols=(0,1))

In [6]:
lat_ma = ma.arange(-46.75,-11,0.5)

In [7]:
lon_ma = ma.arange(136.25,178.5,0.5)

In [8]:
pdsi_np = np.zeros((len(time_ma), len(lon_ma), len(lat_ma)))

In [9]:
pdsi_ma = ma.masked_equal(pdsi_np,0)
ma.set_fill_value(pdsi_ma, 1.e20)

In [10]:
anzda_recon_np = np.array(anzda_recon_txt)
for i in range(len(anzda_pdsi_lon_lat_pair_txt)):
    lon_txt = anzda_pdsi_lon_lat_pair_txt[i][0]
    lat_txt = anzda_pdsi_lon_lat_pair_txt[i][1]
    lon_index = np.where(lon_ma == lon_txt)[0][0]
    lat_index = np.where(lat_ma == lat_txt)[0][0]
    pdsi_ma[:,lon_index,lat_index] = anzda_recon_np[:,i]

In [11]:
nc = netCDF4.Dataset('/d4/yzhou/Drought_atlas/ANZDA/anzda.nc','w')

In [12]:
nc.createDimension('time', size=len(time_ma))

nc.createDimension('longitude', size=len(lon_ma))

nc.createDimension('latitude', size=len(lat_ma))

<type 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 72

In [13]:
times = nc.createVariable("time","f8",("time",))
latitudes = nc.createVariable("latitude","f4",("latitude",))
longitudes = nc.createVariable("longitude","f4",("longitude",))

In [14]:
pdsi = nc.createVariable("pdsi", "f4", ("time", "longitude", "latitude",), fill_value = 1.e20)

In [15]:
import time
import cf_units
nc.title = "Eastern Australia and New Zealand Drought Atlas (ANZDA)"
nc.history = "Original data downloaded from https://www.ncdc.noaa.gov/cdo/f?p=519:1:0::::P1_STUDY_ID:20245. Created " + time.ctime(time.time())
nc.file_author = "Yuxin Zhou yzhou@ldeo.columbia.edu"
nc.reference = "Jonathan G. Palmer, Edward R. Cook, Chris S.M. Turney, Kathy Allen, Pavla Fenwick, Benjamin I. Cook, Alison O'Donnell, Janice Lough, Pauline Grierson, Patrick Baker. 2015. Drought variability in the eastern Australia and New Zealand summer drought atlas (ANZDA, CE 1500-2012) modulated by the Interdecadal Pacific Oscillation. Environmental Research Letters, 10(12). doi: 10.1088/1748-9326/10/12/124002"
latitudes.units = "degrees north"
latitudes.long_name = "latitude"
latitudes.axis = "Y"
longitudes.units = "degrees east"
longitudes.long_name = "longitude"
longitudes.axis = "X"
pdsi.units = cf_units.Unit('1').format()
pdsi.long_name = "Palmer Drought Severity Index DJF"
times.units = "days since 0001-01-01 00:00:00.0"
times.calendar = "gregorian"
times.axis = "T"

In [16]:
latitudes[:] = lat_ma
longitudes[:] = lon_ma
pdsi[:,:,:] = pdsi_ma

In [17]:
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num
dates = [datetime(int(n),1,1) for n in time_np]

In [18]:
times[:] = ma.asarray(date2num(dates, units=times.units,calendar=times.calendar))

In [24]:
times

RuntimeError: NetCDF: Not a valid ID

In [20]:
nc.close()